In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Step 1: Import Libraries
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, r2_score











In [7]:
# Step 2: Load the datasets
diet_df = pd.read_csv('/content/drive/MyDrive/Updated_Diet_dataset.csv')
workout_df = pd.read_csv('/content/drive/MyDrive/Enhanced_Workout_dataset.csv')


In [8]:
# Step 3: Preprocessing for Diet Dataset
# Extract numerical columns for scaling
numerical_columns_diet = ['Price', 'Calories']
scaler_diet = StandardScaler()
diet_df[numerical_columns_diet] = scaler_diet.fit_transform(diet_df[numerical_columns_diet])

# Encode categorical columns in diet dataset
categorical_columns_diet = ['Veg_Non', 'Nutrient', 'Disease', 'Diet', 'User_Preferences', 'Category']
label_encoders_diet = {}
for col in categorical_columns_diet:
    le = LabelEncoder()
    diet_df[col] = le.fit_transform(diet_df[col])
    label_encoders_diet[col] = le


In [9]:
# Step 4: Preprocessing for Workout Dataset
# Impute missing values for Workout dataset using the most frequent strategy
imputer = SimpleImputer(strategy='most_frequent')
workout_df[['Exercise_Type', 'Fitness_Level', 'Previous_Injuries', 'Motivation_Level']] = imputer.fit_transform(
    workout_df[['Exercise_Type', 'Fitness_Level', 'Previous_Injuries', 'Motivation_Level']])

# Extract numerical columns for scaling
numerical_columns_workout = ['Calories_Burn', 'Dream_Weight', 'Actual_Weight', 'Age',
                             'Duration', 'Heart_Rate', 'BMI', 'Exercise_Intensity', 'Motivation_Level']
scaler_workout = StandardScaler()
workout_df[numerical_columns_workout] = scaler_workout.fit_transform(workout_df[numerical_columns_workout])


In [10]:
# Step 5: Fit LabelEncoder on the entire 'Exercise_Type' column BEFORE splitting the data
label_encoders_workout = {}
le_exercise_type = LabelEncoder()
workout_df['Exercise_Type'] = le_exercise_type.fit_transform(workout_df['Exercise_Type'])  # Encode before split
label_encoders_workout['Exercise_Type'] = le_exercise_type


In [11]:
# Step 6: Train Diet Model (Gradient Boosting Classifier)
X_diet = diet_df[['Price', 'Calories', 'Veg_Non', 'Nutrient', 'Disease', 'User_Preferences']]
y_diet = diet_df['Diet']

# Split data into training and test sets
X_train_diet, X_test_diet, y_train_diet, y_test_diet = train_test_split(X_diet, y_diet, test_size=0.2, random_state=42)

# Train the Gradient Boosting Classifier for diet recommendation
gb_diet_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_diet_model.fit(X_train_diet, y_train_diet)

# Evaluate the diet model
y_pred_diet = gb_diet_model.predict(X_test_diet)
diet_accuracy = accuracy_score(y_test_diet, y_pred_diet)
diet_classification_report = classification_report(y_test_diet, y_pred_diet)
print(f"Diet Model - Accuracy: {diet_accuracy}")
print(f"Classification Report:\n{diet_classification_report}")


Diet Model - Accuracy: 0.09175
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        41
           1       0.00      0.00      0.00        38
           2       0.00      0.00      0.00       159
           3       0.00      0.00      0.00        48
           4       0.08      0.01      0.02        75
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        72
           7       0.00      0.00      0.00        38
           8       0.00      0.00      0.00        31
           9       0.00      0.00      0.00        53
          10       0.00      0.00      0.00       111
          11       0.00      0.00      0.00        41
          12       0.00      0.00      0.00       124
          13       0.00      0.00      0.00        36
          14       0.00      0.00      0.00       527
          15       0.00      0.00      0.00       143
          16       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Step 7: Train Workout Model (Gradient Boosting Regressor)
X_workout = workout_df[['Calories_Burn', 'Dream_Weight', 'Actual_Weight', 'Age', 'Duration', 'BMI', 'Exercise_Intensity']]
y_workout = workout_df['Exercise_Type']

# Split data into training and test sets
X_train_workout, X_test_workout, y_train_workout, y_test_workout = train_test_split(X_workout, y_workout, test_size=0.2, random_state=42)

# Train the Gradient Boosting Regressor for workout recommendation
gb_workout_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_workout_model.fit(X_train_workout, y_train_workout)

# Evaluate the workout model
y_pred_workout = gb_workout_model.predict(X_test_workout)
workout_mae = mean_absolute_error(y_test_workout, y_pred_workout)
workout_r2 = r2_score(y_test_workout, y_pred_workout)
print(f"Workout Model - MAE: {workout_mae}, R² Score: {workout_r2}")


Workout Model - MAE: 1.0150381219275495, R² Score: 0.0031716200609800405


In [13]:
# Step 8: Create Recommendation System
def recommend_diet_workout(age, weight, height, veg_nonveg, preference):
    """
    Function to recommend a suitable diet and workout based on user input.
    """
    # BMI Calculation
    bmi = weight / ((height / 100) ** 2)

    # Recommend diet
    diet_input = [[0, bmi, veg_nonveg, preference, 0, preference]]  # Adjust to match diet features
    predicted_diet = gb_diet_model.predict(diet_input)[0]
    diet_name = label_encoders_diet['Diet'].inverse_transform([predicted_diet])[0]
    print(f"Recommended Diet: {diet_name}")

    # Recommend workout based on diet recommendation
    workout_input = [[0, weight, weight, age, 30, bmi, preference]]  # Adjust inputs based on model needs
    predicted_workout = gb_workout_model.predict(workout_input)[0]

    # Convert the predicted workout into a suitable label
    predicted_workout_int = round(predicted_workout)  # Since it's regression, round to the nearest integer
    workout_name = label_encoders_workout['Exercise_Type'].inverse_transform([predicted_workout_int])[0]

    print(f"Recommended Workout: {workout_name}")


In [17]:
# Step 9: Example user input for recommendation
age = 25
weight = 70
height = 175
# Assuming 'Veg_Non' was originally encoded as strings ('Veg', 'Non-Veg'),
# and 1 represents 'Veg', we need to transform it accordingly.
# FIX: Check if 'Veg' is in the classes_ of the LabelEncoder
# If not, manually assign the desired encoded value (1 in this case)
if 'Veg' in label_encoders_diet['Veg_Non'].classes_:
    veg_nonveg = label_encoders_diet['Veg_Non'].transform(['Veg'])[0]
else:
    veg_nonveg = 1  # Manually assign 1 for 'Veg' if it's unseen
preference = 2  # Example user preference encoded

# Get recommendations
recommend_diet_workout(age, weight, height, veg_nonveg, preference)

Recommended Diet:  low_fat_diet low_carb_diet vegan_diet hormone_diet high_protien_diet type_a_diet
Recommended Workout: flexibility


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
